In [ ]:
!pip install git+https://github.com/alibaba/TinyNeuralNetwork.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/alibaba/TinyNeuralNetwork.git to /tmp/pip-req-build-g1xn63iw
  Running command git clone -q https://github.com/alibaba/TinyNeuralNetwork.git /tmp/pip-req-build-g1xn63iw
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 109 kB 32.1 MB/s 
     |████████████████████████████████| 3.2 MB 54.4 MB/s 
     |████████████████████████████████| 546 kB 71.1 MB/s 
  Created wheel for TinyNeuralNetwork: filename=TinyNeuralNetwork-0.1.0.20220929234957+a521163753e4b201a4a31d3cb82e5aa1fe328d93-py3-none-any.whl size=352165 sha256=ba1a2123e198bf2ee6eeba0e5e3958f3b157e59834516551511832d9970114c1
  Stored in directory: /tmp/pip-ephem-wheel-cache-0boymxof/wheels/64/6b/f5/9cf69de054ba0de53e572bc2f13988a664f3dc9623e6f0825d
Successfully built TinyNeuralNetwork


In [ ]:
!pip install git+https://github.com/openai/whisper.git 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-1w73cw51
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-1w73cw51
     |████████████████████████████████| 4.9 MB 28.3 MB/s 
     |████████████████████████████████| 163 kB 68.6 MB/s 
     |████████████████████████████████| 6.6 MB 53.1 MB/s 
  Created wheel for whisper: filename=whisper-1.0-py3-none-any.whl size=1175043 sha256=004d3775858273de0d73c74151a159e6d09bfb73a12d618a755a99b23435e525
  Stored in directory: /tmp/pip-ephem-wheel-cache-40rbi8q5/wheels/16/15/89/1c7bb31bd0006793a95549d04785121a8a36daad9158e1e43a
Successfully built whisper


In [ ]:
!git clone https://github.com/usefulsensors/openai-whisper.git

Cloning into 'openai-whisper'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 39 (delta 18), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (39/39), done.


## Convert from pytorch to onnx

In [ ]:
import whisper
import torch

tiny_model = whisper.load_model("tiny")
torch.onnx.export(tiny_model.encoder, torch.randn(1,80,3000).to("cuda"), "/content/whisper-encoder.onnx")
torch.onnx.export(tiny_model.decoder, (torch.tensor([[50258]]).to("cuda"), torch.randn(1,384,384).to("cuda")), "/content/whisper-decoder_main.onnx")
torch.onnx.export(tiny_model.decoder, (torch.tensor([[50258, 50259, 50359]]).to("cuda"), torch.randn(1, 384, 384).to("cuda")), "/content/whisper-decoder_language.onnx")

####Encoder TFLite model

In [ ]:
!pip install onnx-tf

from onnx_tf.backend import prepare
import onnx

onnx_model_path = '/content/whisper-encoder.onnx'
tf_model_path = 'model_tf-encoder'

onnx_model = onnx.load(onnx_model_path)
tf_rep = prepare(onnx_model)
tf_rep.export_graph(tf_model_path)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 226 kB 28.2 MB/s 
     |████████████████████████████████| 1.1 MB 60.9 MB/s 
     |████████████████████████████████| 13.1 MB 59.3 MB/s 


In [ ]:
import tensorflow as tf

saved_model_dir = 'model_tf-encoder'
tflite_model_path = 'whisper-encoder.tflite'

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert()

# Save the model
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

##Decoder TFLite model

In [ ]:
from onnx_tf.backend import prepare
import onnx

onnx_model_path = '/content/whisper-decoder_main.onnx'
tf_model_path = 'model_tf-decoder_main'

onnx_model = onnx.load(onnx_model_path)
tf_rep = prepare(onnx_model)
tf_rep.export_graph(tf_model_path)

In [ ]:
import tensorflow as tf

saved_model_dir = 'model_tf-decoder_main'
tflite_model_path = 'whisper-decoder_main.tflite'

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert()

# Save the model
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

##Decoder language TFLite model

In [ ]:
from onnx_tf.backend import prepare
import onnx

onnx_model_path = '/content/whisper-decoder_language.onnx'
tf_model_path = 'model_tf-decoder_language'

onnx_model = onnx.load(onnx_model_path)
tf_rep = prepare(onnx_model)
tf_rep.export_graph(tf_model_path)

In [ ]:
import tensorflow as tf

saved_model_dir = 'model_tf-decoder_language'
tflite_model_path = 'whisper-decoder_language.tflite'

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert()

# Save the model
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

## Convert from pytorch to tflite using Tinynn

In [ ]:
import whisper
import torch

import torch.nn as nn
from tinynn.converter import TFLiteConverter
from tinynn.graph.quantization.quantizer import PostQuantizer
from tinynn.graph.tracer import model_tracer
from tinynn.util.train_util import DLContext, get_device, train

model = whisper.load_model("tiny")
model.eval()
audio = whisper.load_audio("/content/openai-whisper/filename.mp3")
audio = whisper.pad_or_trim(audio)
print(audio.shape)
# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)
print(mel.shape)
quantizer = PostQuantizer(model, mel, work_dir='out', config={'asymmetric': True, 'per_tensor': False})
#qat_model = quantizer.quantize()


(480000,)
torch.Size([80, 3000])
